<a href="https://colab.research.google.com/github/dylanesq/2-Months-Project-LLM/blob/main/LlamaIndexV2_DeepSeek_R1_Distill_Qwen_1_5B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 10.8 MB/s eta 0:00:00


#Loading

 **SIMPLE DIRECTORY READER**
READING PDFS

In [3]:

from llama_index.core import SimpleDirectoryReader

reader=SimpleDirectoryReader(input_dir="/content/drive/MyDrive/002MONTHSPROJECT/", required_exts=[".pdf"])
document_1=reader.load_data()

# Chunking Randomly
[TokenTextSplitter](https://docs.llamaindex.ai/en/v0.10.17/api/llama_index.core.node_parser.TokenTextSplitter.html#tokentextsplitter)

In [4]:
from llama_index.core.node_parser import TokenTextSplitter
splitter = TokenTextSplitter(
    chunk_size=1024,
    chunk_overlap=100,
    separator=" ",
)

*For Review Purpose*

In [5]:
token_nodes = splitter.get_nodes_from_documents(
    document_1, show_progress=True
)

Parsing nodes:   0%|          | 0/9 [00:00<?, ?it/s]

# LLM Setup
*   [HuggingFaceLLM](https://docs.llamaindex.ai/en/v0.9.48/api_reference/llms/huggingface.html#huggingfacellm)
*   [Llama 3.2 3B
](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct)



update : logging without CLI *(contains exposed HF Token , how to fix)*

In [6]:
# @title LOGIN INFO [ HIDDEN]
!huggingface-cli login --token HF_TOKEN --add-to-git-credential

Token is valid (permission: fineGrained).
The token `llama3forAnkan` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llama3forAnkan`


In [7]:
!huggingface-cli whoami
#!huggingface-cli logout

ankandata


#LLM Init.

In [8]:
!pip install llama_index.llms.huggingface
from llama_index.llms.huggingface import HuggingFaceLLM

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 27.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

system prompt

In [9]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""


Quantization [ 8 bit Working]

[HF Bits&Bytesconfig](https://huggingface.co/docs/transformers/en/main_classes/quantization#transformers.BitsAndBytesConfig)
[Bitnbytes docu](https://pypi.org/project/bitsandbytes/https://)

[Why Quantize](https://medium.com/@techresearchspace/what-is-quantization-in-llm-01ba61968a51)

In [10]:
!pip install -U bitsandbytes
!pip install accelerate
#!pip install --upgrade transformers accelerate bitsandbytes
from transformers import BitsAndBytesConfig

import torch
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
#quantization_config = BitsAndBytesConfig(load_in_8bit=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.2 MB/s eta 0:00:00


In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id)
model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Embedding(151665, 1536)

In [12]:
tokenizer

LlamaTokenizerFast(name_or_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', vocab_size=151643, model_max_length=16384, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<｜end▁of▁sentence｜>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<｜end▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<｜User｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151645: AddedToken("<｜Assistant｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151646: AddedToken("<｜begin▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151647: AddedToken("<|EOT|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151648: AddedToken("<think>", rstrip=False

In [ ]:
#print("After Quantisation :"(llm.get_memory_footprint())/1e9) need to rework this

# **Embedding & Vectorization**


Vector Imports

In [13]:
from llama_index.core import VectorStoreIndex
#NOT REQUIRED SimpleDirectoryReader, ServiceContext  #Vector store index is for indexing the vector


Embedding Imports
* [HuggingFaceEmbeddings](https://docs.llamaindex.ai/en/stable/examples/embeddings/huggingface/)  
*   [LAngchain HF Embedding](https://python.langchain.com/api_reference/huggingface/embeddings/langchain_huggingface.embeddings.huggingface.HuggingFaceEmbeddings.html#huggingfaceembeddings)
*   [SBERT](https://www.sbert.net/)
*   [Llamindex embedding](https://docs.llamaindex.ai/en/stable/api_reference/embeddings/huggingface/)





In [14]:
!pip install llama-index-embeddings-huggingface
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Set up the HuggingFaceEmbedding class with the required model to use with llamaindex core.
embed_model  = HuggingFaceEmbedding(model_name = "sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Wrapper [not working without it]

In [18]:
llm = HuggingFaceLLM(
    model=model,
    tokenizer=tokenizer,
    #generate_kwargs={"max_new_tokens": 256},
)

# Service Context/Settings

Set Up
Edit  : ServiceContext is deprecated.

*   Use llama_index.settings.Settings instead
*   [Migration to Settings Llama Index](https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/service_context_migration/)


In [19]:
# from llama_index.core import ServiceContext this is obsolote
from llama_index.core import Settings
Settings.llm=llm
Settings.embed_model=embed_model
Settings.node_parser=splitter

#Indexing

In [20]:

index=VectorStoreIndex.from_documents(document_1)#no longer required to have servicecontext
#query_engine=index.as_query_engine(topK=5)

#Query Engine



*   [Querying LlamaIndex](https://docs.llamaindex.ai/en/stable/module_guides/querying/)
*   [Query Engine Module
](https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/modules/)


Custom Query engine **[working]**

In [21]:

from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
#from llama_index.core.indices.postprocessor import SimilarityPostprocessor

retriever=VectorIndexRetriever(index=index,similarity_top_k=5)
#postprocessor=SimilarityPostprocessor(similarity_cutoff=0.60)

query_engine=RetrieverQueryEngine(retriever=retriever)
#query_engine=RetrieverQueryEngine(retriever=retriever,node_postprocessors=[postprocessor])

**Vanilla Query Response**

In [22]:
prompt="what is this document about?"

Responses

In [23]:
response=query_engine.query(prompt)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [ ]:
norank_retrieved_nodes = retriever.retrieve(prompt)
print(f"Retrieved {len(norank_retrieved_nodes)} nodes:")
for i, node in enumerate(norank_retrieved_nodes):
    print(f"\n--- Node {i + 1} ---")
    print(node.get_content())
    print(f"Score: {node.score}")

In [24]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)
#print(response)

Final Response: <think> Okay, so I need to figure out what this
document is about based on the given context information. Let me read
through the information provided carefully.  First, I see that the
document is titled "Tanmay Bhat - Wikipedia.pdf." That's a standard
title, so it's likely that the document is a Wikipedia article about
Tanmay Bhat. Wikipedia articles usually provide an overview of the
topic, so I can start there.  Looking at the page labels, I see they
go from 1 to 8, which suggests there are multiple pages within the
same document. Each page likely covers different aspects of Tanmay
Bhat's background, career, and contributions.  Starting with page
label 1, which is "Tanmay Bhat: Wikipedia." I notice that the page has
several sections. The first section is a list of his co-founders,
Aditya Kulshrestha, Nishant Tanwar, Rohan Joshi, Zakir Khan, Vishal
Dayama, and Piyush Sharma. These are referred to as their "codenames."
So, this section explains who he worked with befor

# Implement Rerank
[Query Bundle](https://docs.llamaindex.ai/en/v0.10.17/api/llama_index.core.schema.QueryBundle.html#querybundle)

In [25]:
#from llama_index.core.retrievers import VectorIndexRetriever (Already Imported)
from llama_index.core import QueryBundle
from llama_index.core.indices.postprocessor import LLMRerank

#from typing import List

retriever2 = VectorIndexRetriever(
        index=index,
        similarity_top_k=5,
        #vector_store_query_mode="default",

    )

query_bundle = QueryBundle(prompt)

retrieved_nodes = retriever2.retrieve(query_bundle)

reranker = LLMRerank(
            choice_batch_size=5,
            top_n=5,
        )
retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )

#pprint_response(retrieved_nodes,show_source=True)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [26]:
query_engine2=RetrieverQueryEngine(retriever=retriever2)

In [ ]:
print(f"Retrieved {len(retrieved_nodes)} nodes:")
for i, node in enumerate(retrieved_nodes):
    print(f"\n--- Node {i + 1} ---")
    print(node.get_content())
    print(f"Score: {node.score}")

In [27]:
response2=query_engine2.query(query_bundle)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [28]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response2,show_source=True)
#print(response)

Final Response: <think> Okay, I need to figure out what this document
is about based on the provided context information and the given
query. Let me start by reading through the context carefully.  The
document is titled "Tanmay Bhat - Wikipedia.pdf" and it's located at a
specific file path on the user's drive. The page labels are 1 through
25, each pointing to different parts of the document. The context
information includes various sections discussing Tanmay Bhat's
background, his work in media, his relationship with other people, and
some controversies.  First, I'll look at the main sections. The first
part talks about Tanmay Bhat's background, mentioning his work in the
TV series "Bakchod" and his role as a comedian. It also mentions his
role as the 'OP' in Desi YouTube, along with others like Aditya
Kulshrestha, Nishant Tanwar, and others.  Next, there's a section
about his "OG gang," which includes Aditya, Nishant, Rohan, Zakir, and
Vishal. This group has their own codenames and 